In [1]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
from connection_keys import connect_bq_client
import statsmodels.api as sm
import os
import warnings
import itertools
warnings.filterwarnings("ignore")

In [2]:
client = connect_bq_client()

sql = """
    SELECT *
    FROM `db_market_product.gpu`
    ORDER BY dt_carga asc
"""

df = client.query(sql).to_dataframe()
df['data'] = df.dt_carga.dt.date
df['ds_nome_produto'] = df['ds_nome_produto'].str.upper()

In [3]:
df.sample(10)

,ds_marca,ds_nome_produto,vl_preco_atual,vl_preco_com_desconto,vl_porcentagem_do_desconto,qt_quantidade_estoque,oferta,ds_disponibilidade,loja,dt_carga,data
371121,Colorful,PLACA DE VIDEO COLORFUL IGAME GEFORCE RTX 3060...,3159.00,2685.15,15.0,0,None,False,Kabum,2022-12-04 10:01:25.534657+00:00,2022-12-04
360922,Zotac,PLACA DE VIDEO ZOTAC GEFORCE GT 710 2GB GDDR5 ...,313.53,313.53,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-12-02 10:01:28.931946+00:00,2022-12-02
57252,Gigabyte,PLACA DE VIDEO GIGABYTE RADEON RX 560 2GB GDDR...,605.81,605.81,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-09 23:32:58.961206+00:00,2022-10-09
214137,PCYES,PLACA DE VÍDEO PCYES AMD RADEON RX580 8GB GDRR...,4170.00,3753.00,10.0,0,None,False,Kabum,2022-11-05 10:02:01.888203+00:00,2022-11-05
183747,Gigabyte,PLACA DE VIDEO GIGABYTE GEFORCE GTX 1070 8GB G...,2272.59,2272.59,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-30 10:02:00.607904+00:00,2022-10-30
22654,EVGA,PLACA DE VIDEO EVGA GEFORCE GTX 1080 8GB GDDR5...,2894.30,2894.30,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-03 10:02:34.041786+00:00,2022-10-03
168675,Asus,PLACA DE VÍDEO ASUS TUF NVIDIA RTX 3090 TI - 2...,15569.90,14791.41,5.0,0,None,False,Kabum,2022-10-27 22:02:29.876756+00:00,2022-10-27
19522,MSI,PLACA DE VIDEO MSI GEFORCE RTX 3080 VENTUS 3X ...,9214.76,9214.76,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-02 23:15:42.944356+00:00,2022-10-02
278937,MSI,PLACA DE VIDEO MSI RADEON RX 480 GAMING X 8G,1873.84,1873.84,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-17 10:01:27.435768+00:00,2022-11-17
20427,Gigabyte,"PLACA DE VÍDEO GIGABYTE GTX 1050TI, 4GB, GDDR5...",2687.90,2687.90,0.0,0,None,False,Kabum,2022-10-02 23:16:41.074157+00:00,2022-10-02


In [147]:
df.iloc[df.index == 187149]['ds_nome_produto'].tolist() # ds_nome_produto_resumido ds_nome_produto

['PLACA DE VÍDEO EVGA NVIDIA GEFORCE RTX 3060 XC BLACK GAMING 12G P5 3655 KR, LHR,  12GB GDDR6, DLSS, RAY TRACING - 12G-P5-3655-KR']

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384100 entries, 0 to 384099
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   ds_marca                    384100 non-null  object             
 1   ds_nome_produto             384100 non-null  object             
 2   vl_preco_atual              384100 non-null  float64            
 3   vl_preco_com_desconto       384100 non-null  float64            
 4   vl_porcentagem_do_desconto  384100 non-null  float64            
 5   qt_quantidade_estoque       142715 non-null  Int64              
 6   oferta                      384100 non-null  object             
 7   ds_disponibilidade          384100 non-null  object             
 8   loja                        384100 non-null  object             
 9   dt_carga                    384100 non-null  datetime64[ns, UTC]
 10  data                        384100 non-null 

In [4]:
df.groupby(['dt_carga'])['loja'].count()
df.groupby(['data', 'loja'])['dt_carga'].nunique()

data        loja  
2022-09-29  Kabum     2
            Pichau    2
2022-09-30  Kabum     2
            Pichau    2
2022-10-01  Kabum     2
                     ..
2022-12-04  Pichau    2
2022-12-05  Kabum     2
            Pichau    2
2022-12-06  Kabum     1
            Pichau    1
Name: dt_carga, Length: 138, dtype: int64

In [3]:
# df_datas_loja = df.groupby(['data', 'loja'])['dt_carga'].nunique()
# df_datas_loja
maxDf = df.groupby(['data'])['dt_carga'].max()
minDf = df.groupby(['data'])['dt_carga'].min()
maxDates = maxDf.tolist()
df = df.loc[df['dt_carga'].isin(maxDates)]

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
# nltk.download('popular') # Rodar apenas uma vez para fazer a instalação


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/igorferreira/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/igorferreira/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/igorferreira/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/igorferreira/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/igorferreira/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/igorferreira/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.

True

In [10]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent
ex = "PLACA DE VÍDEO ASUS NVIDIA GEFORCE GTX 1050 TI OC CERBERUS, 4GB GDDR5 - CERBERUS-GTX1050TI-O4G"
sent = preprocess(ex)
sent
pattern = 'NP: {<NNP>*<CD>*<NNP>}'
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

(S
  (NP
    PLACA/NNP
    DE/NNP
    VÍDEO/NNP
    ASUS/NNP
    NVIDIA/NNP
    GEFORCE/NNP
    GTX/NNP
    1050/CD
    TI/NNP)
  (NP OC/NNP CERBERUS/NNP)
  ,/,
  (NP 4GB/CD GDDR5/NNP)
  -/:
  CERBERUS-GTX1050TI-O4G/NN)


In [13]:
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
iob_tagged = tree2conlltags(cs)
pprint(iob_tagged)
ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(ex)))
print(ne_tree)

[('PLACA', 'NNP', 'B-NP'),
 ('DE', 'NNP', 'I-NP'),
 ('VÍDEO', 'NNP', 'I-NP'),
 ('ASUS', 'NNP', 'I-NP'),
 ('NVIDIA', 'NNP', 'I-NP'),
 ('GEFORCE', 'NNP', 'I-NP'),
 ('GTX', 'NNP', 'I-NP'),
 ('1050', 'CD', 'I-NP'),
 ('TI', 'NNP', 'I-NP'),
 ('OC', 'NNP', 'B-NP'),
 ('CERBERUS', 'NNP', 'I-NP'),
 (',', ',', 'O'),
 ('4GB', 'CD', 'B-NP'),
 ('GDDR5', 'NNP', 'I-NP'),
 ('-', ':', 'O'),
 ('CERBERUS-GTX1050TI-O4G', 'NN', 'O')]
(S
  (ORGANIZATION PLACA/NNP)
  (PERSON DE/NNP)
  VÍDEO/NNP
  ASUS/NNP
  NVIDIA/NNP
  GEFORCE/NNP
  GTX/NNP
  1050/CD
  TI/NNP
  OC/NNP
  CERBERUS/NNP
  ,/,
  4GB/CD
  (ORGANIZATION GDDR5/NNP)
  -/:
  CERBERUS-GTX1050TI-O4G/NN)


In [5]:
def get_resume_product_name(name):
    # string = df['nome_produto'][df['id'] == 259].item()
    pattern = "(RTX|GTX|GT|RX|WX|Quadro|R7|R5|R9|HD)\s(\d+)"
    resume_name = re.search(pattern, name)
    if resume_name is None:
        return ''
    else:
        return resume_name.group()

df['ds_nome_produto_resumido'] = df.apply(lambda x: get_resume_product_name(x['ds_nome_produto']), axis=1)

In [17]:
def get_resume_product_name(name):
    pattern = "(RTX|GTX|GT|RX|WX|QUADRO|R7|R5|R9|HD)(.?.?)((\w*)\d+)( ?(TI|XT|X|SUPER))?"
    resume_name = re.search(pattern, name)
    if resume_name is None:
        return ''
    else:
        return resume_name.group().strip()

df['ds_nome_produto_resumido'] = df.apply(lambda x: get_resume_product_name(x['ds_nome_produto']), axis=1)

In [21]:
df.sample(10)

,ds_marca,ds_nome_produto,vl_preco_atual,vl_preco_com_desconto,vl_porcentagem_do_desconto,qt_quantidade_estoque,oferta,ds_disponibilidade,loja,dt_carga,data,ds_nome_produto_resumido
149056,Evga,"PLACA DE VÍDEO EVGA NVIDIA GEFORCE GT 730 2GB,...",588.22,499.99,15.0,974,"[{'ds_oferta_nome': 'Next Level', 'vl_oferta_p...",True,Kabum,2022-10-23 22:01:33.563930+00:00,2022-10-23,GT 730
86406,EVGA,PLACA DE VIDEO EVGA GEFORCE GTX 1060 6GB GDDR5...,1537.48,1537.48,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-13 22:39:18.005597+00:00,2022-10-13,GTX 1060
201872,MSI,PLACA DE VIDEO MSI GEFORCE RTX 2070 ARMOR 8G O...,3180.64,3180.64,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-02 22:02:37.754569+00:00,2022-11-02,RTX 2070
50554,Galax,PLACA DE VIDEO 1GB GT 710 D3 64B GALAX 71GGF4D...,305.90,305.90,0.0,0,None,False,Kabum,2022-10-08 22:16:42.262354+00:00,2022-10-08,GT 710
79366,PNY,PLACA DE VIDEO PNY GEFORCE RTX 3060 TI XLR8 GA...,4886.25,4886.25,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-12 22:39:14.465550+00:00,2022-10-12,RTX 3060 TI
353938,Asus,PLACA DE VIDEO ASUS GEFORCE RTX 3080 TI OC 12G...,14544.32,8863.63,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-30 22:01:35.228910+00:00,2022-11-30,RTX 3080 TI
152299,Gainward,"PLACA DE VIDEO GAINWARD, RTX 3060 TI GHOST 8GB...",3589.00,3589.00,0.0,0,None,False,Kabum,2022-10-24 22:03:24.470733+00:00,2022-10-24,RTX 3060 TI
170924,Gigabyte,"PLACA DE VÍDEO GIGABYTE AMD RADEON RX 6750XT, ...",4117.64,3499.99,15.0,0,None,False,Kabum,2022-10-27 22:02:29.876756+00:00,2022-10-27,RX 6750XT
141964,PNY,PLACA DE VIDEO PNY QUADRO K2200 4GB GDDR5 128-...,2499.00,2499.00,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-10-22 22:01:36.053461+00:00,2022-10-22,QUADRO K2200
304726,PNY,PLACA DE VIDEO PNY GEFORCE RTX 3060 XLR8 REVEL...,5271.59,5271.59,12.0,<NA>,None,OUT_OF_STOCK,Pichau,2022-11-21 22:01:39.589281+00:00,2022-11-21,RTX 3060 X


# Transpostar para outra lugar depois

In [11]:
# Limpa o nome de arquivos RTX A2000, 12GB
import os
import re
folder = './teste/'
for filename in os.listdir('./teste'):
    nome = re.sub(r'\([^)]*\)', '', filename)
    nome = re.sub(r'\s+(?=[.,?!])', '', nome)
    os.rename(os.path.join(folder, filename), os.path.join(folder, nome))